# Tabulation of Fractional Cover data within tiled shapefile polygons

**What does this notebook do?**

This notebook is a pilot collaboration between Geoscience Australia and Australian Bureau of Statistics. The purpose of the notebook is to use a shapefile polygon boundaries to load fractional cover dataset, complete zonal statistics and tabulate the results.

**Requirements**

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles`

`module load dea/20181213`


**Background**

Data from the Landsat 5,7 and 8 satellite missions are accessible through Digital Earth Australia (DEA). The code snippets in this notebook will let you retrieve and plot the Fractional Cover (FC25) data stored in DEA.


**How to use this notebook**

A basic understanding of any programming language is desirable but one doesn't have to be an expert Python programmer to manipulate the code to get and display the data.This doc applies to the following Landsat satellites, Fractional Cover bands and the WOfS dataset:

- Landsat 5
- Landsat 7
- Landsat 8
- PV - Photosythetic vegetation
- NPV - Non-Photosythetic vegetation
- BS - Bare Soil
- UE - Unmixing Error
- Water Observations from Space (WOFs)
- WOfS Feature Layer (WOFL)

**Bugs still to fix**

- Qaulity metrics for tiled/large SA2
- Check area of SA2 for Percentage calculation


**Errors or bugs**

If you find an error or bug in this notebook, please contact erin.telfer@ga.gov.au.


## 1. Import Libraries

In [1]:
%matplotlib inline

from datetime import time, datetime
import os.path

from matplotlib import pyplot as plt
import pandas as pd
import numpy
import csv
import xarray as xr
import rasterio
import rasterio.features
import fiona
from datetime import datetime
import dask
from dask.delayed import delayed
from dask.distributed import LocalCluster, Client
import tempfile

import datacube
from datacube import Datacube
from datacube.virtual import construct, construct_from_yaml
from datacube.ui.task_app import year_splitter
from datacube.utils.geometry import CRS

print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-05-07 16:19:16


## 2. Define directories and years of interest

In [4]:
#Set folder  and SA2 shapefile locations
shapefile_path = '/g/data/r78/ext547/abs/input/SA_2016_clipped_albers_3577.shp'
output_path = '/g/data/r78/ext547/abs/output/'
name_of_output_file ='tabulate_FC_large_SA2'

In [7]:
#Specify time years of interest
start_year, end_year = 2000, 2002
time_range = (str(start_year), str(end_year))
print(time_range)

('2000', '2002')


## 3. Set up a local dask cluster
Some calculations take more memory than is available on a system.  By breaking the data up into chunks, we can chain a sequence of operations together, and work on the data a small piece at a time.

This lets several processes work at the same time, and manage total memory usage for the calculations.

In more advanced setups, we can distribute the work across multiple computers, using all of their memory and CPU power.

* We set `n_workers` to be the number of worker applications we want to run in the background, doing the processing of the chunk-based steps we have chained together.
* The `mem_per_worker` setting defines how much memory at most is available to each of the workers.
* `chunk_size` sets the width and height of the chunk in pixels of the size will break up the data into.

VDI has 8 CPUs available, and a total of 32GB of memory, but you will typically be sharing those with several (2-10) other users.   

In [8]:
#Set up dask cluster
n_workers = 7
threads_per_worker=1
mem_per_worker = 8e9  # 8e9 is 8GB (8,000,000,000 bytes)

chunk_size = {'time': 1, 'x': 2000, 'y': 2000}

In [9]:
cluster = LocalCluster(local_dir=tempfile.gettempdir(), 
                       n_workers=n_workers, 
                       threads_per_worker=threads_per_worker,
                       memory_limit=mem_per_worker)
client = Client(cluster)
dask.config.set(get=client.get)
client

Client Scheduler: tcp://127.0.0.1:32861 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 7 Cores: 7 Memory: 56.00 GB


We also get a dashboard to see how the system is running, by clicking the link above after the cell has been run.

## 4. Connect to the Datacube 

In [10]:
dc = Datacube()
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-05-07 16:19:53


## 5. Construct the virtual product

In [11]:
#Remove Landsat 7 scenes with the Scan Line Correction (SLC) missing data
LS7_BROKEN_DATE = datetime(2003, 5, 31)
is_pre_slc_failure = lambda dataset: dataset.center_time < LS7_BROKEN_DATE
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-05-07 16:19:53


In [12]:
#Create function to ensure wofls in correct format
def wofls_fuser(dest, src):
    where_nodata = (src & 1) == 0
    numpy.copyto(dest, src, where=where_nodata)
    return dest
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-05-07 16:19:53


In [13]:
#Create virtual product so that datacube data can be loaded effectively within memory
fc_and_water_yaml = """
        juxtapose:
          - collate:
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls5_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls5_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls7_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                        # dataset_predicate: __main__.is_pre_slc_failure
                      - transform: make_mask
                        input:
                            product: ls7_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls8_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls8_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
          - transform: make_mask
            input:
                product: wofs_albers
                group_by: solar_day
                fuse_func: __main__.wofls_fuser
            flags:
                wet: true
            mask_measurement_name: water
"""
fc_and_water = construct_from_yaml(fc_and_water_yaml)

## 6. Set up functions

In [14]:
def geometry_mask(geoms, geobox, all_touched=False, invert=False, chunks=None):
    """
    Create a mask from shapes.

    By default, mask is intended for use as a
    numpy mask, where pixels that overlap shapes are False.
    :param list[Geometry] geoms: geometries to be rasterized
    :param datacube.utils.GeoBox geobox:
    :param bool all_touched: If True, all pixels touched by geometries will be burned in. If
                             false, only pixels whose center is within the polygon or that
                             are selected by Bresenham's line algorithm will be burned in.
    :param bool invert: If True, mask will be True for pixels that overlap shapes.
    """
    data = rasterio.features.geometry_mask([geom.to_crs(geobox.crs) for geom in geoms],
                                           out_shape=geobox.shape,
                                           transform=geobox.affine,
                                           all_touched=all_touched,
                                           invert=invert)
    if chunks is not None:
        data = dask.array.from_array(data, chunks=tuple(chunks[d] for d in geobox.dims))
        
    coords = [xr.DataArray(data=coord.values, name=dim, dims=[dim], attrs={'units': coord.units}) 
              for dim, coord in geobox.coords.items()]
    return xr.DataArray(data, coords=coords)
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-05-07 16:19:53


In [15]:
def get_shapes(shape_file):
    """
    Extract spatial inforamtion from polygons within shapefile
    """
    with fiona.open(shape_file) as shapes:
        crs = datacube.utils.geometry.CRS(shapes.crs_wkt)
        for shape in shapes:
            if shape['geometry'] is None:
                continue
            geom = datacube.utils.geometry.Geometry(shape['geometry'], crs=crs)
            geom = geom.to_crs(CRS('EPSG:3577'))
            yield geom, shape['properties']
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-05-07 16:19:53


In [16]:
def fc_and_water_summary(data, mask_int):
    """
    Calculate the percentage and the area of each FC component, 
    water, and unclassified data within each SA2 boundary. 
    """
    # Where there is water, set the FC bands to 0%
    valid_data = numpy.isfinite(data['BS'])
    fc = data[['PV', 'NPV', 'BS']].where(data.water==0, other=0)
    fc['water'] = data.water.where(valid_data) * numpy.float32(100)
    fc = fc.apply(lambda data_array: data_array.clip(0, 100).where(valid_data))

    has_data = valid_data.groupby(valid_data.time.astype('datetime64[M]'), squeeze=False).sum(dim='time', skipna=False)
    has_data = has_data.sum(dim=['x','y'], skipna=True) / (mask_int / 100)
    
    # Flatten to a monthly mean
    fc = fc.groupby(fc.time.astype('datetime64[M]'), squeeze=False).mean(dim='time', skipna=True)
    # Calculate percentage of cover based on area of mask
    percentage = fc.sum(dim=['x','y']) * (100 / mask_int)
    for da in percentage.data_vars.values():
        da.attrs['units'] = '%'

    fc_sum = fc.sum(dim=['x','y'])
    fc_sum = fc_sum.PV + fc_sum.NPV + fc_sum.BS + fc_sum.water
    
    unclass_size = mask_int - fc_sum
    unclass_percentage = unclass_size * (100 / mask_int)
    fc['unclassified'] = unclass_percentage 
    percentage['unclassified'] = unclass_percentage
    
#     print(f"   PV = {int(percentage.PV.values)}%, NPV = {int(percentage.NPV.values)}%, BS = {int(percentage.BS.values)}%, water = {int(percentage.water.values)}%, unclassified = {int(unclass_percentage)}%")
           
    pixel_area_in_metres2 = 25 * 25
    m2_to_km2 = (1 / 1_000_000)
    percent_to_fraction = (1 / 100)
    
    area = (fc * (pixel_area_in_metres2 * m2_to_km2 * percent_to_fraction)).sum(dim=['x','y'])
    area = area.rename({'BS': 'BS_area', 
                        'PV': 'PV_area', 
                        'NPV': 'NPV_area', 
                        'water': 'water_area',
                        'unclassified':'unclassified_area'})
    
    for da in area.data_vars.values():
        da.attrs['units'] = 'km^2'
    fc = percentage.merge(area)
    fc['average_data_count'] = has_data
    return fc

In [17]:
def plot_stacked(ds, sa2_id,plot_title='title', show=True):
    """
    Create and save a stacked plot to visualise FC components
    """
    if not show:
        plt.ioff()
        
    fig,ax = plt.subplots(figsize=(10,5))
    ax.stackplot(ds.dropna(dim='time').time.data, 
                 ds.dropna(dim='time').PV,
                 ds.dropna(dim='time').NPV, 
                 ds.dropna(dim='time').BS, 
                 ds.dropna(dim='time').water, 
                 ds.dropna(dim='time').unclassified, 
                 colors = ['darkolivegreen','olive','tan','darkblue','red'], 
                 labels=['PV','NPV','BS','Water','Unclassified',])
    plt.legend(loc='upper center', ncol = 5)
    plt.title(f'FC components: SA2 ID {sa2_id}', size=12)
    plt.ylabel('Percentage (%)', size=12) #Set Y label
    plt.xlabel('Date', size=12) #Set X label
    
    plt.savefig(f'{output_path}/{sa2_id}_{plot_title}.png');
    plt.close(fig)
    
    # Turn interactive back on
    if not show:
        plt.show()
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-05-07 16:19:53


In [18]:
def month_splitter(start_year, end_year_inclusive):
    """ 
    Split specified years into months 
    """
    yield from (str(p) for p in pd.period_range(start=pd.Period(start_year).start_time, 
                               end=pd.Period(end_year_inclusive).end_time, 
                               freq='M'))

In [19]:
def output_csv(input_ds, sa2_id, sa2_name, sa2_size, albers_id, monthly_or_annual='frequency'):
    """
    Save tabulated data into a csv
    """
    input_ds = input_ds.to_dataframe()
    input_ds.insert(0,'SA2_id', sa2_id)
    input_ds.insert(1,'SA2_name', sa2_name)
    input_ds.insert(2,'albers_id', albers_id)
    input_ds.insert(3,'SA2_size', sa2_size)
    input_ds.to_csv(f"{output_path}/{name_of_output_file}_{monthly_or_annual}.csv",mode='a',header=False)  

## 7. Set up the query
For each year and polygon query the product, apply the geometry mask and compute the fractional cover stats

Using `client.compute()` lets us use the monthly results in calculating the annual results at the same time.

In [20]:
#Obtain spatial information from shapefile
shape_file = os.path.expanduser(f'{shapefile_path}')
shapes = list(get_shapes(shape_file))

# #Specify a particular SA2 boundary, if required
#shapes = [(g,p) for g, p in shapes if str(p['SA2_MAIN16']) == '801111141']
shapes = [(g,p) for g, p in shapes if str(p['AREASQKM16']) < '25_000']


print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-05-07 16:21:06


If we have enough resources, we can start the query and calculation of the next year's data while the previous is still being calculated. `by_slice=False` will be faster, but use more memory.

For larger areas `by_slice` will need to be `True`, so that the compute cluster does not become overwhelmed.  

If you get the error:
> `distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting`

then you will need to set `by_slice=True`

In [21]:
by_slice=True

In [22]:
monthly_values = None
def process_area(geometry, sa2_id, sa2_name, sa2_size, albers_id, time_range):
    monthly_values = []
    annual_values = []
    mask = None
    
    # Virtual Products have a bug handling geometry objects, use this instead of `geopolygon=geometry` for now
    search_terms = {
        'x': (geometry.envelope.left, geometry.envelope.right),
        'y': (geometry.envelope.top, geometry.envelope.bottom),
        'crs': str(geometry.crs),
    }  

    for sub_time_range in month_splitter(time_range[0], time_range[-1]):
        print(sub_time_range)
        try:
            data = fc_and_water.load(dc, dask_chunks=chunk_size, 
                                     time=sub_time_range, 
                                     **search_terms)
        except ValueError:
            print(f'    No data for {sub_time_range} , skipping...')
            continue

        if mask is None:
            mask = geometry_mask([geometry], data.geobox, invert=True, chunks=data.chunks)
            mask_int = mask * 1
            mask_int = mask_int.sum() * 100
            mask_int.load()

        data = data.where(mask)
        monthly_data = fc_and_water_summary(data, mask_int)
        monthly_data = client.compute(monthly_data, sync=by_slice)
        monthly_values.append(monthly_data)
        
    if not by_slice:
        print("  all years queried, hard load data")
        monthly_values = client.gather(monthly_values)
        
    monthly_values = xr.concat(monthly_values, dim='time')
    
    monthly_qa = xr.Dataset(coords={'time': monthly_values.time})
    monthly_qa['has_values'] = (('time',), [1] * monthly_values.time.size)
    monthly_qa['has_10'] = monthly_values['unclassified'] <= 10
    
    monthly_values = monthly_values.where(monthly_values['unclassified'] < 10).dropna(dim='time')
    
    
    annual_values = monthly_values.resample(time='1YS').mean(dim='time', skipna=True)
    
    # Replace the mean of the monthly obs count with the annual total obs count
    annual_avg_data_count = monthly_values['average_data_count'].resample(time='1YS').sum(dim='time', skipna=True)
    annual_values['average_data_count'] = annual_avg_data_count
    
    annual_qa = monthly_qa.resample(time='1YS').sum(dim='time', skipna=True)
    annual_qa = 12 - annual_qa
    annual_values = annual_values.merge(annual_qa)
    
    print(f"Calculation complete for annual values")
    
    plot_stacked(monthly_values, sa2_id, plot_title='monthly_plot_wofs',show=False)
    plot_stacked(annual_values, sa2_id, plot_title='annual_plot_wofs', show=False)
    
    print("All data loaded, save to csv")
    output_csv(monthly_values, sa2_id, sa2_name, sa2_size, monthly_or_annual='monthly')
    output_csv(annual_values, sa2_id, sa2_name, sa2_size,albers_id, monthly_or_annual='annual')
       
    print(f"SA2 {sa2_id} done")

## 8. Process query

In [23]:
#Create empty CSV with specified headings
header = ['DATE','SA2_MAIN16', 'SA2_NAME16', 'ALBERS3577_ID', 'AREASQKM16',
          'PV_PERCENTAGE','NPV_PERCENTAGE','BS_PERCENTAGE','WOFL_PERCENTAGE',
          'UNCLASSIFIED_PERCENTAGE','PV_AREA_SQKM_ALBERS3577','NPV_AREA_SQKM_ALBERS3577',
          'BS_AREA_SQKM_ALBERS3577','WOFL_AREA_SQKM_ALBERS3577','UNCLASSIFIED_AREA_SQKM_ALBERS3577', 
          'AVERAGE_OBSERVATION_COUNT']

additional_annual_header = ['MONTHS_WITH_NO_DATA', 'MONTHS_WITH_LOW_DATA']

with open(f"{output_path}/{name_of_output_file}_annual.csv","w") as outcsv: #create csv to save output and add header text
    writer = csv.writer(outcsv)
    writer.writerow(header + additional_annual_header)
    
with open(f"{output_path}/{name_of_output_file}_monthly.csv","w") as outcsv: #create csv to save output and add header text
    writer = csv.writer(outcsv)
    writer.writerow(header)

In [24]:
#Run tabulation script
for geometry, properties in shapes:
    sa2_id = str(properties['SA2_MAIN16'])
    sa2_name = str(properties['SA2_NAME16'])
    sa2_size = str(properties['AREASQKM16'])
    albers_id = str(properties['label'])
    print(f"SA2 ID: {sa2_id}, Albers tile: {albers_id}, SA2 size: {sa2_size}km^2, time: {time_range}")
    
    try:
        process_area(geometry, sa2_id, sa2_name, sa2_size, albers_id, time_range)
    except Exception as e:
          print(f"Could not process {sa2_id}: {e}")
          client.restart()

SA2 ID: 105011095, Albers tile: 14,-34, SA2 size: 21216.0023km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 105011095 done
SA2 ID: 105011095, Albers tile: 13,-35, SA2 size: 21216.0023km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 105011095 done
SA2 ID: 105011095, Albers tile: 14,-35, SA2 size: 21216.0023km^2, time: 

2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
    No data for 2002-09 , skipping...
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 105021098 done
SA2 ID: 105021098, Albers tile: 8,-35, SA2 size: 146690.0605km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 105021098 done
SA2 ID: 105021098, Albers tile: 9,-35, SA2 size: 146690.0605km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
20

2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 113011257 done
SA2 ID: 113011257, Albers tile: 13,-38, SA2 size: 23020.2043km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 113011257 done
SA2 ID: 113011257, Albers tile: 11,-39, SA2 size: 23020.2043km^2, time: ('2000', '2002')
2000-01
2000-02
    No data for 2000-02 , skipping...
2000-03
2000-04
2000-05
20

2000-02
    No data for 2000-02 , skipping...
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
    No data for 2002-06 , skipping...
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 312011340 done
SA2 ID: 312011340, Albers tile: 16,-25, SA2 size: 21092.2365km^2, time: ('2000', '2002')
2000-01
2000-02
    No data for 2000-02 , skipping...
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
    No data for 2002-06 , skipping...
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 312011340 done
SA2 ID: 315011396

2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 315011396 done
SA2 ID: 315011396, Albers tile: 14,-16, SA2 size: 112960.6967km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 315011396 done
SA2 ID: 315011396, Albers tile: 10,-17, SA2 size: 112960.6967km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-1

...

2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 702011052 done
SA2 ID: 702011052, Albers tile: 2,-27, SA2 size: 129516.4673km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 702011052 done
SA2 ID: 702011052, Albers tile: 3,-27, SA2 size: 129516.4673km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10


2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 702011053 done
SA2 ID: 702011053, Albers tile: -1,-22, SA2 size: 192633.3183km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 702011053 done
SA2 ID: 702011053, Albers tile: 0,-22, SA2 size: 192633.3183km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
    No data for 2000-03 , skipping...
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2

2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 702011053 done
SA2 ID: 702011053, Albers tile: -3,-26, SA2 size: 192633.3183km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
    No data for 2001-03 , skipping...
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 702011053 done
SA2 ID: 702011053, Albers tile: -2,-26, SA2 size: 192633.3183km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03


2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 702051068 done
SA2 ID: 702051068, Albers tile: -3,-17, SA2 size: 133608.5533km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 702051068 done
SA2 ID: 702051068, Albers tile: -2,-17, SA2 size: 133608.5533km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-1

2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 702051068 done
SA2 ID: 702051068, Albers tile: -1,-20, SA2 size: 133608.5533km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
2000-04
2000-05
2000-06
2000-07
2000-08
2000-09
2000-10
2000-11
2000-12
2001-01
2001-02
    No data for 2001-02 , skipping...
2001-03
2001-04
2001-05
2001-06
2001-07
2001-08
2001-09
2001-10
2001-11
2001-12
2002-01
2002-02
2002-03
2002-04
2002-05
2002-06
2002-07
2002-08
2002-09
2002-10
2002-11
2002-12
Calculation complete for annual values
All data loaded, save to csv
SA2 702051068 done
SA2 ID: 702051068, Albers tile: 0,-20, SA2 size: 133608.5533km^2, time: ('2000', '2002')
2000-01
2000-02
2000-03
    No data for 2

## Combine results for tiled SA2

In [25]:
#read area results of previous workflow combine tiles for each SA2
# Note, quality metrics aren't included at this point in time

df = pd.read_csv(f"{output_path}/{name_of_output_file}_annual.csv", header=0)
df = df [['DATE','SA2_MAIN16', 'SA2_NAME16','ALBERS3577_ID', 'AREASQKM16','PV_AREA_SQKM_ALBERS3577','NPV_AREA_SQKM_ALBERS3577',
          'BS_AREA_SQKM_ALBERS3577','WOFL_AREA_SQKM_ALBERS3577','UNCLASSIFIED_AREA_SQKM_ALBERS3577']]
df = df.groupby(['SA2_MAIN16','DATE','SA2_NAME16','AREASQKM16']).sum()
df = df.reset_index()

In [26]:
#calculate percentage of each SA2
df['PV_PERCENTAGE'] = df['PV_AREA_SQKM_ALBERS3577']/df['AREASQKM16'] *100
df['NPV_PERCENTAGE'] = df['NPV_AREA_SQKM_ALBERS3577']/df['AREASQKM16'] *100
df['BS_PERCENTAGE'] = df['BS_AREA_SQKM_ALBERS3577']/df['AREASQKM16'] *100
df['WOFL_PERCENTAGE'] = df['WOFL_AREA_SQKM_ALBERS3577']/df['AREASQKM16'] *100
df['UNCLASSIFIED_PERCENTAGE'] = df['UNCLASSIFIED_AREA_SQKM_ALBERS3577']/df['AREASQKM16'] *100

In [27]:
df

,SA2_MAIN16,DATE,SA2_NAME16,AREASQKM16,PV_AREA_SQKM_ALBERS3577,NPV_AREA_SQKM_ALBERS3577,BS_AREA_SQKM_ALBERS3577,WOFL_AREA_SQKM_ALBERS3577,UNCLASSIFIED_AREA_SQKM_ALBERS3577,PV_PERCENTAGE,NPV_PERCENTAGE,BS_PERCENTAGE,WOFL_PERCENTAGE,UNCLASSIFIED_PERCENTAGE
0,105011095,2000-01-01,Nyngan - Warren,21216.0023,8970.763112,9217.812429,2572.204275,55.294490,0.000079,42.283004,43.447452,12.123888,0.260626,3.741123e-07
1,105011095,2001-01-01,Nyngan - Warren,21216.0023,6128.063057,10826.842977,3910.153276,17.808606,0.000064,28.884155,51.031494,18.430208,0.083940,3.031713e-07
2,105011095,2002-01-01,Nyngan - Warren,21216.0023,4017.252592,10087.296567,6721.875838,8.054584,0.000069,18.935012,47.545699,31.683046,0.037965,3.230898e-07
3,105021098,2000-01-01,Far West,146690.0605,27321.934350,56437.627070,58249.166310,1655.111929,0.000304,18.625621,38.474064,39.709007,1.128305,2.069262e-07
4,105021098,2001-01-01,Far West,146690.0605,16004.570921,56991.324320,69905.706170,639.449177,0.000317,10.910467,38.851524,47.655380,0.435919,2.159376e-07
5,105021098,2002-01-01,Far West,146690.0605,10954.972661,49896.174130,82652.583150,273.964701,0.000278,7.468108,34.014693,56.345047,0.186764,1.892437e-07
6,113011257,2000-01-01,Griffith Region,23020.2043,9067.163757,10207.598801,3202.911497,167.354290,0.000094,39.387851,44.341912,13.913480,0.726989,4.066446e-07
7,113011257,2001-01-01,Griffith Region,23020.2043,5631.446175,11788.416591,5052.930954,164.017134,0.000087,24.463059,51.209001,21.949983,0.712492,3.783067e-07
8,113011257,2002-01-01,Griffith Region,23020.2043,3787.212666,10696.731169,8136.972720,77.133373,0.000075,16.451690,46.466708,35.347092,0.335068,3.261717e-07
9,215021398,2000-01-01,Mildura Region,21569.5292,5818.219560,8910.012470,6121.274016,20.328720,0.000103,26.974254,41.308331,28.379266,0.094247,4.792807e-07


In [28]:
#save as csv
df.to_csv(f"{output_path}/{name_of_output_file}_annual_large_sa2.csv")